In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os

import pandas as pd
import numpy as np

Mounted at /content/drive


In [2]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)
def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]

In [3]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/projects/Dacon/covid19_diagnostics/"
createFolder(folder_path + "LB_Archive/Ensemble")

In [13]:
lb_scores = [i.split("_")[1] for i in diff(os.listdir(folder_path + "LB_Archive/"), ["Ensemble"])]
print(lb_scores)

['0.59890', '0.58635', '0.58788', '0.60210', '0.5648963908', '0.5816651754', '0.59026', '0.6029795763', '0.579594', '0.5398390542', '0.5241859274', '0.4423366096', '0.579100064', '0.5742642326', '0.5795402375', '0.5886606495', '0.5898549341', '0.5821918305', '0.5945039683', '0.605429376', '0.5960159141', '0.5824564153', '0.0647442873', '0.5827515013', '0.5827581338', '0.5832926131']


In [25]:
threshold = 0.07859

# TOP N Prediction

In [38]:
top_k = 3
createFolder(folder_path + "LB_Archive/Ensemble/top_" + str(top_k))
ensemble_path = folder_path + "LB_Archive/Ensemble/top_" + str(top_k)

In [39]:
lb_scores_ordered = sorted(lb_scores, reverse=True)[:top_k]
print(lb_scores_ordered)

['0.605429376', '0.6029795763', '0.60210']


In [40]:
submission = pd.read_csv(folder_path + "open.zip (Unzipped Files)/sample_submission.csv")
submission.iloc[:,1] = 0

In [41]:
info_list = []
for i in lb_scores_ordered:
    tmp_pred = [i for i in os.listdir(folder_path + "LB_Archive/LB_" + str(i) + "/") if i.split("_")[0] == "testPred"][0]
    info_list.append((tmp_pred, i))
    submission.iloc[:,1] += pd.read_csv(folder_path + "LB_Archive/LB_" + str(i) + "/" + tmp_pred).iloc[:,1] / top_k

In [42]:
submission

,id,covid19
0,3806,0.061952
1,3807,0.038914
2,3808,0.040597
3,3809,0.040519
4,3810,0.051918
...,...,...
5727,9533,0.042583
5728,9534,0.087672
5729,9535,0.059239
5730,9536,0.039502


In [43]:
submission.iloc[:,1] = [1 if i > threshold else 0 for i in submission.iloc[:,1]]
submission.head()

,id,covid19
0,3806,0
1,3807,0
2,3808,0
3,3809,0
4,3810,0


In [44]:
submission.to_csv(ensemble_path + "/testPred_top_" + str(top_k) + ".csv", index=False)

In [45]:
info_list

[('testPred_mfcc_cat_gbm_try1.csv의 사본', '0.605429376'),
 ('testPred_mfcc_polyFeature_lgb_goss_try1.csv의 사본', '0.6029795763'),
 ('testPred_mfcc_ageOnly4_lgb_goss_thresholdOri_tuningPosWeight_try1.csv의 사본',
  '0.60210')]

In [46]:
with open(ensemble_path + "/info.txt", "w") as f:
    for i in info_list:
        f.write(i[0] + " : " + str(i[1]) + "\n")